In [4]:
import tensorflow as tf

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [58]:
import os, shutil

In [9]:
print(os.getcwd())

/Users/alessandrobiagi/Desktop/Kaggle_Datasets/Kaggle_Bradipi/FoodRecognition/scripts


In [22]:
def creating_dataset_directories(type_dir, category):
    new_cat_dir = os.path.join(type_dir, category)
    os.mkdir(new_cat_dir)


In [11]:
# Directory where you'll store your dataset
base_dir = '../src/food-101/'

# Directories for the training, validation, and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [24]:
categories = ['sashimi', 'spaghetti_bol']
type_dirs = [train_dir, validation_dir, test_dir]

for category in categories:
    for type_dir in type_dirs:
        creating_dataset_directories(type_dir=type_dir,
                                     category=category)

In [69]:
sashimi_dir = base_dir + 'images/sashimi'
spaghetti_bol_dir = base_dir + 'images/spaghetti_bolognese'

sashimi_names = os.listdir(sashimi_dir)
sashimi_full_names = [os.path.join(sashimi_dir, x) for x in sashimi_names]

spaghetti_bol_names = os.listdir(spaghetti_bol_dir)
spaghetti_bol_full_names = [os.path.join(spaghetti_bol_dir, x) for x in spaghetti_bol_names]


In [38]:
print(len(sashimi_names))
print(len(spaghetti_bol_names))

1000
1000


In [42]:
sashimi_train_names = sashimi_full_names[0:700]
sashimi_val_names = sashimi_full_names[700:850]
sashimi_test_names = sashimi_full_names[850:100]

In [70]:
spaghetti_bol_train_names = spaghetti_bol_full_names[0:700]
spaghetti_bol_val_names = spaghetti_bol_full_names[700:850]
spaghetti_bol_test_names = spaghetti_bol_full_names[850:1000]

In [64]:
def creating_datasets(dataset_names, dst_dir, type_dir, category):
    for x in dataset_names:
        img_name = x.split("/")[-1]
        dst_dir_type = os.path.join(dst_dir, type_dir)
        dst_dir_full = os.path.join(dst_dir_type, category)
        dst = os.path.join(dst_dir_full, img_name)
        shutil.copy(x, dst)

In [76]:
base_dst_dir_dataset = '../src/food-101'
sashimi_names_list = [sashimi_train_names, sashimi_val_names, sashimi_test_names]
spaghetti_bol_list = [spaghetti_bol_train_names, spaghetti_bol_val_names, spaghetti_bol_test_names]
type_dataset = ['train', 'validation', 'test']
categories_dataset = ['sashimi', 'spaghetti_bol']

In [ ]:
for category in categories_dataset:
    if category == 'sashimi':
        dataset_names_list = sashimi_names_list
    elif category == 'spaghetti_bol':
        dataset_names_list = spaghetti_bol_list
    fo

In [68]:
creating_datasets(dataset_names=sashimi_train_names,
                 dst_dir='../src/food-101/',
                 type_dir='train',
                 category='sashimi')

In [ ]:
creating_datasets(dataset_names=sashimi_val_names,
                 dst_dir='../src/food-101/',
                 type_dir='validation',
                 category='sashimi')

In [ ]:
creating_datasets(dataset_names=sashimi_test_names,
                 dst_dir='../src/food-101/',
                 type_dir='test',
                 category='sashimi')

In [71]:
creating_datasets(dataset_names=spaghetti_bol_train_names,
                 dst_dir='../src/food-101/',
                 type_dir='train',
                 category='spaghetti_bol')

In [73]:
creating_datasets(dataset_names=spaghetti_bol_val_names,
                 dst_dir='../src/food-101/',
                 type_dir='validation',
                 category='spaghetti_bol')

In [74]:
creating_datasets(dataset_names=spaghetti_bol_test_names,
                 dst_dir='../src/food-101/',
                 type_dir='test',
                 category='spaghetti_bol')

In [77]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [78]:
train_dir

'../src/food-101/train'

In [85]:
train_generator = train_datagen.flow_from_directory(
                                train_dir,
                                target_size=(150, 150), # resize all images to 150 x 150
                                batch_size=20,
                                class_mode='binary')

Found 1400 images belonging to 2 classes.


In [86]:
validation_generator = test_datagen.flow_from_directory(
                                validation_dir,
                                target_size=(150, 150),
                                batch_size=20,
                                class_mode='binary')

Found 300 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=70,
                epochs=30,
                validation_data=validation_generator,
                validation_steps=15)